In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys, os
#import mlflow
#import mlflow.azureml

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.26.0


In [2]:
from azureml.core import Workspace
ws = Workspace.from_config(path="config.json")
print(ws.name, ws.location, ws.resource_group, sep='\t')

WS-JCROSELLO-P7	westeurope	cloud-shell-storage-westeurope


In [3]:
# see workspace details
ws.get_details()

{'id': '/subscriptions/75c51acd-24dc-4674-9a83-dd6319951bf8/resourceGroups/cloud-shell-storage-westeurope/providers/Microsoft.MachineLearningServices/workspaces/WS-JCROSELLO-P7',
 'name': 'WS-JCROSELLO-P7',
 'identity': {'principal_id': '9cc66bc7-ee73-42a6-b9af-e123e8e136d0',
  'tenant_id': '31f99866-f9ca-40e6-bc44-16e162aed2f1',
  'type': 'SystemAssigned'},
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'ae81e516-8e98-48db-8aa2-005fbca07768',
 'sdkTelemetryAppInsightsKey': '6d7b2b55-6d18-4126-a3ee-251f78cfa6e5',
 'description': '',
 'friendlyName': 'WS-JCROSELLO-P7',
 'creationTime': '2021-04-14T08:30:38.6122892+00:00',
 'containerRegistry': '/subscriptions/75c51acd-24dc-4674-9a83-dd6319951bf8/resourceGroups/cloud-shell-storage-westeurope/providers/Microsoft.ContainerRegistry/registries/ae81e5168e9848db8aa2005fbca07768',
 'keyVault': '/subscriptions/75c51acd-24dc-4674-9a83-dd6319951bf8/resourcegroups/cl

In [4]:
%%writefile train.py



from azureml.core import Run
import os


from urllib import request
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import decomposition
from sklearn import preprocessing
sns.set()
# Python program to read 
# json file 
 
import json 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.optimizers import TFOptimizer
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
from keras.initializers import Constant
import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stemming = PorterStemmer()
Lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words("english"))
stops2 = set(stopwords.words("french"))

import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.



import matplotlib.pyplot as plt
#%matplotlib inline


# Get the experiment run context
run = Run.get_context()





print("Prepare the dataset...")
# Prepare the dataset
commentData = pd.read_csv('P7_01_0_training.1600.processed.noemoticon.csv', encoding='ISO-8859-1')
df = commentData
train_df = df.drop(["id","date","flag","flag","user"], axis=1)
train_df = train_df.rename(columns={"target": "Sentiment"})

def clean_str(x):
    
    x = str(x)
   
    # Convert to lower case
    text = x.lower()
    
    # Tokenize
    tokens = nltk.word_tokenize(text)
    
    # Keep only words (removes punctuation + numbers)
    # use .isalnum to keep also numbers
    token_words = [w for w in tokens if w.isalpha()]
    
    # Stemming
    #stemmed_words = [stemming.stem(w) for w in token_words]

    # lemmatizer    
    lemmatizer_words = [Lemmatizer.lemmatize(w) for w in token_words]
    
    # Remove stop words
    meaningful_words = [w for w in token_words if not w in stops]
    
    # Remove stop words
    meaningful_words2 = [w for w in meaningful_words if not w in stops2]   
    
    # Rejoin meaningful stemmed words
    joined_words = ( " ".join(meaningful_words2))

    
    # Return cleaned data
    return joined_words


train_df['text'] = train_df['text'].apply(clean_str)

df = train_df
df_0 = df[df['Sentiment'] == 0].sample(frac=1)
df_4 = df[df['Sentiment'] == 4].sample(frac=1)

# we want a balanced set for training against - there are 7072 `0` examples
sample_size = min(len(df_0), len(df_4))

data = pd.concat([df_0.head(sample_size), df_4.head(sample_size)]).sample(frac=1)
sentences = data['text']
tokenizer = Tokenizer(num_words = 4000)
tokenizer.fit_on_texts(sentences)
sequence = tokenizer.texts_to_sequences(sentences)

import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

index_of_words = tokenizer.word_index
embed_num_dims = 100
max_seq_len = 1000
padded_seq = pad_sequences(sequence , maxlen = max_seq_len )
Y = pd.get_dummies(data['Sentiment']).values

X_train,X_test,Y_train,Y_test = train_test_split(padded_seq,Y ,train_size = 0.55)

# Glove
f = open('glove.6B.100d.txt', encoding="utf8")
embedd_index = {}
for line in f:
    val = line.split()
    word = val[0]
    coff = np.asarray(val[1:],dtype = 'float')
    embedd_index[word] = coff

f.close()

embedding_matrix = np.zeros((len(index_of_words) + 1, embed_num_dims))

tokens = []
labels = []

for word,i in index_of_words.items():
    temp = embedd_index.get(word)
    if temp is not None:
        embedding_matrix[i] = temp
        
#for plotting
        tokens.append(embedding_matrix[i])
        labels.append(word)

#Embedding layer before the actaul BLSTM 
embedd_layer = Embedding(len(index_of_words) + 1 , embed_num_dims , input_length = max_seq_len , weights = [embedding_matrix])

print("Building model...")
# Bidirectional LSTM model avec GLOVE 
model = Sequential()
model.add(embedd_layer)
model.add(Bidirectional(LSTM(30 , return_sequences = True , dropout = 0.1 , recurrent_dropout = 0.1)))
model.add(GlobalMaxPooling1D())
model.add(Dense(30,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation = 'sigmoid'))

# Train LSTM model
add = Adam(lr = 0.01)
model.compile(loss = 'categorical_crossentropy' , optimizer = add , metrics = ['accuracy'])
hist = model.fit(X_train,Y_train,epochs = 5 , batch_size = 100, validation_data = (X_test,Y_test))

score = model.evaluate(X_test,Y_test, batch_size= 100, verbose=1)
print("Test score:", score[0])
print("Test accuracy:", score[1])
print("Accuracy: %.2f%%" % (score[1] * 100))
# Save the model 
#for heavy model architectures, .h5 file is unsupported.
weigh= model.get_weights();    pklfile= "./model1.pkl"
try:
    fpkl= open(pklfile, 'wb')    #Python 3     
    pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
    fpkl.close()
except:
    fpkl= open(pklfile, 'w')    #Python 2      
    pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
    fpkl.close()


from tensorflow.python.keras.optimizers import TFOptimizer
import tensorflow as tf
model_path = "./model"
tf.keras.models.save_model(model, model_path, overwrite=True, include_optimizer=True, save_format=None)

import onnxmltools

onnx_model = onnxmltools.convert_keras(model) 

onnxmltools.utils.save_model(onnx_model, 'model.onnx')

# Save the model 
from tensorflow.python.keras.optimizers import TFOptimizer
import tensorflow as tf
model_path = "./model.h5"
tf.keras.models.save_model(model, model_path, overwrite=True, include_optimizer=True, save_format='h5')

run.complete()


Overwriting train.py


In [5]:
import train

C:\Users\jcrro\anaconda3\envs\EnvP7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jcrro\anaconda3\envs\EnvP7\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jcrro\anaconda3\envs\EnvP7\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jcrro\anaconda3\envs\EnvP7\lib\site-packages\tensorflow\python\f

Prepare the dataset...
Building model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 880 samples, validate on 720 samples
Epoch 1/5
880/880 [==============================] - 11s 13ms/step - loss: 0.6754 - accuracy: 0.5841 - val_loss: 0.6504 - val_accuracy: 0.6333
Epoch 2/5
880/880 [==============================] - 11s 13ms/step - loss: 0.5040 - accuracy: 0.7625 - val_loss: 0.6112 - val_accuracy: 0.7014
Epoch 3/5
880/880 [==============================] - 11s 13ms/step - loss: 0.2820 - accuracy: 0.8886 - val_loss: 0.8649 - val_accuracy: 0.6639
Epoch 4/5
880/880 [==============================] - 11s 13ms/step - loss: 0.1142 - accuracy: 0.9568 - val_loss: 0.9297 - val_accuracy: 0.6833
Epoch 5/5
720/720 [==============================] - 2s 3ms/step
Test score: 1.168547261092398
Test accuracy: 0.6958333253860474
Accuracy: 69.58%


The ONNX operator number change on the optimization: 22 -> 16


In [6]:
import os
import urllib.request
from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "model.h5",
                       model_name = "my_model",
                       description = "P7 Analyse sentiments1-Déploiement du Modèle sur mesure avancé",
                       workspace = ws)

Registering model my_model


In [7]:
from azureml.core.model import Model
model = Model(ws, 'my_model', version=1)

In [8]:
model

Model(workspace=Workspace.create(name='WS-JCROSELLO-P7', subscription_id='75c51acd-24dc-4674-9a83-dd6319951bf8', resource_group='cloud-shell-storage-westeurope'), name=my_model, id=my_model:1, version=1, tags={}, properties={})

In [9]:
%%writefile score.py


import numpy as np
import pandas as pd
import os
import pickle
import json
import time
from azureml.core.model import Model
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_seq_len = 1000

# Called when the deployed service starts
def init():
    global model

    
 
    # load models
    #model = Model.get_model_path(model_name = 'my_model')
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './model.h5')
    model = model = load_model(model_path)    
    

# Handle requests to the service
def run(data):
    try:
        data = json.loads(data)    
        tokenizer = Tokenizer(num_words = 100)   
        _seq = tokenizer.texts_to_sequences(data['text'])
        x_test = pad_sequences(_seq, maxlen=max_seq_len)
        result = np.mean(np.array([[float(x)] for x in model.predict([x_test])[0]]))
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [10]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("tensorflow")
conda_dep.add_pip_package("nltk")
with open("myenv.yml","w") as f:
    f.write(conda_dep.serialize_to_string())
# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

# ACI

In [11]:
import os
import urllib.request
from azureml.core.model import Model
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
# Register the model to deploy



# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = ws,
    name = "myservice",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-16 10:59:40+02:00 Creating Container Registry if not exists.
2021-04-16 10:59:41+02:00 Registering the environment.
2021-04-16 10:59:42+02:00 Use the existing image.
2021-04-16 10:59:43+02:00 Generating deployment configuration.
2021-04-16 10:59:44+02:00 Submitting deployment to compute..
2021-04-16 10:59:51+02:00 Checking the status of deployment myservice..
2021-04-16 11:05:02+02:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [12]:
print(service.state)

Healthy


## Test du modèle web service

In [23]:
import requests
import json

scoring_uri = 'http://0adf4c9f-4bbd-4167-9a23-dca73c35ba35.westeurope.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}


test_data = json.dumps({'text': 'today is a great day'})

response = requests.post(scoring_uri, test_data, headers=headers)
print(response.status_code)

print(response.json())

print(response.text)

200
0.33497992157936096
0.33497992157936096


In [24]:
service.delete()